In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u

import gastronomy as gas

# Get theoretical gas-phase photoelectric absorption cross-sections

Uses the constants from [Verner & Yaklovev (1995)](https://ui.adsabs.harvard.edu/abs/1995A%26AS..109..125V/abstract)

In [ ]:
ENERGY = np.arange(0.1, 10.0, 0.005) * u.keV
ELEMENT = 'Si'

In [ ]:
atomic_number = gas.abundances.Z[ELEMENT]
ion_state = 0 # number of electrons removed from the ion (0=neutral)
shell = 1 # principle quantum number for the shell (1 = K shell, 2 = L shell)

In [ ]:
cross_section = gas.VY1995.compute_xsect(ENERGY, atomic_number, ion_state, shell) # returns units of Mbarn

In [ ]:
YUNIT = 'cm^2'
plt.plot(ENERGY, cross_section.to(YUNIT))
plt.xlabel(ENERGY.unit)
plt.ylabel(YUNIT)

## How do you get from cross-section to a value for MaxTau?

Remember that in astronomy, we talk about the total number of atom integrated along the line of sight in terms of "column density" -- that is, a number per unit area (cm$^{-2}$).

We can quantify how much light interacts with the total line-of-sight amount of the element of interest by multiplying the cross-section for interaction by the column density, yielding a unitless value. We call this value the "optical depth" and it is typically represented with the Greek letter $\tau$:

$$ \tau = \sigma (E) {\rm N}_{\rm Z}$$

where $\sigma (E)$ is the cross-sectional area (as a funciton of energy) and ${\rm N}_{\rm Z}$ is the column density of the element of interest with atomic number $Z$.

The fraction of light that is removed as a result of absorption by element $Z$ is:

$$ 1 - e^{-\tau} $$

With the XSPEC model, "edge", the $e^{-\tau}$ factor is computed for you. When you fit for "MaxTau" in the XSPEC "edge" model, you are getting the amplitude of the photoelectric jump that you see in the plot, above, multiplied by the column density.

**Thought exercise:** How would you determine the abundance of an element from the XSPEC ''MaxTau'' value, assuming the Verner & Yaklovev (1996) model?

## Let's predict the MaxTau value, given some abundance table

The gastronomy library has some of my favorite abundance tables stored in the `gastronomy.abundances` module. By default, all of the functions built-in to this model use the Wilms et al. (2000) ISM abundances table.

Here I show how to get the total predicted column density of an element, from this table.

In [ ]:
NH = 1.e22 * u.Unit('cm^-2') # column density of H-nuclei, as a benchmark

In [ ]:
Si_per_H = gas.abundances.get_total_abund(ELEMENT) # returns the number of Fe atoms expected per H nucleus in the ISM
NSi = Si_per_H * NH

In [ ]:
print(NSi)

**Thought question:** How would you compute the number of expected Si atoms when you have the abundance of a different element, like Ne?

With the column density of my element of interest, I can now plot optical depth as a function of energy.

In [ ]:
plt.plot(ENERGY, (cross_section * NSi).to(''))
plt.xlabel(ENERGY.unit)
plt.ylabel(r'$\tau$ (gas absorption)')

**Thought question:** What is the expected MaxTau value for this sight line? Did I need to plot it in order to figure out this value?

**Coding assignment:** Can you write a function that computes the MaxTau value for a given element (Z, assume neutral) and shell (1 = K shell and 2 = L shell)? Can you make a plot of the MaxTau value as it is expected to change with increasing Ne abundance? How about increasing H abundance?

**Science question:** The dominant ion state for some elements in the cold ISM are not-neutral. For example, Si in the cold component of the ISM is expected to be singly ionized (Si$^{+1}$ in chemistry notation, SiII in astronomy). Does increasing the ion number from 0 to 1 change the MaxTau values significantly? Check for Ne, Si, Mg, and Fe L shell.